In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
# # プログレスバーのラベル設定
tqdm_notebook.pandas(desc="progress: ")

In [2]:
df = pd.read_excel('sw_text_190206.xlsx')
df

,left_id,strength_left,weakness_left,right_id,strength_right,weakness_right
0,1,実のところ「AとBに差がない」とは「BはAで無いとは言えない」を意味します。消極的かつ回りく...,いいかえると「Bの結果がAであるにしては逸脱している」かどうかで差の有無を結論するわけです。...,4,αとは「真実として差がないにも関わらず、差があると結論する」偽陽性の確率,βとは「真実として差があるにも関わらず、差がないと結論する」偽陰性の確率
1,2,00人がCVR10%の現LPに接触した場合の入会数は20人を中心にばらつきます。そして面積の...,Bの入会数を計測し、その結果がこの赤い部分に該当する場合は「差がある」と結論すると決めたなら...,5,もしもBの真実のCVRが期待してたような15%でなくAと同様の10%である場合、Bの計測後の...,なお、決めたと表現したように、αを何%するかは恣意的です。赤い面積が小さくなるようにαを設定...
2,3,在籍メンバーの8割がエンジニア,営業部隊を持っておらず、売上が鈍化,6,強固な営業組織を持ち全国に支社を持つ,開発部隊を持っておらず拡大が急務


In [3]:
'''形態素解析結果をarray形式で出力する関数'''
def get_wakati_text_array(text):
    #np.nanがstr型ではなくfloat型とされるためstr型以外はneologdn.normalizeメソッドを実施しないように条件分岐
        if not isinstance(text, str):
            pass
        else:            
            text_normalization = neologdn.normalize(text)
            # 大文字英字を小文字英字に変更
            text_normalization = text_normalization.lower()    
            neologd_tagger = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
            wakati_text = neologd_tagger.parse(text_normalization).replace('\n', '\t').split("\t")[:-2]
            wakati_results_array = np.array([wakati_text]).reshape(-1, 6)
            
            return wakati_results_array



'''array形式で出力された形態素解析結果をデータフレーム型としてリストに格納、そのリストを縦に連結したデータフレームとして出力する関数'''
def create_wakati_df(wakati_array, col):
    
    wakati_list = [pd.DataFrame(i) for i in wakati_array]
    
    array_df_list = []
    for a, b in zip(df[col], wakati_list):
        array_df_list.append(b.assign(id = a))
        
        wakati_df = pd.concat(array_df_list, axis=0)\
    .rename(columns={0:'surface', 1:'yomi', 2:'original', 3:'type', 4:'katsuyoukei', 5:'katsuyougata'})
    
    return wakati_df

In [4]:
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
s_left_wakati_array = df['strength_left'].progress_apply(get_wakati_text_array)

s_left_wakati_df = create_wakati_df(s_left_wakati_array, 'left_id')

s_left_wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,id
0,実,ジツ,実,接頭詞-名詞接続,,,1
1,のところ,ノトコロ,能取湖,名詞-固有名詞-一般,,,1
2,「,「,「,記号-括弧開,,,1
3,a,a,a,名詞-固有名詞-組織,,,1
4,と,ト,と,助詞-並立助詞,,,1


In [5]:
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
w_left_wakati_array = df['weakness_left'].progress_apply(get_wakati_text_array)

w_left_wakati_df = create_wakati_df(w_left_wakati_array, 'left_id')

w_left_wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,id
0,いいかえる,イイカエル,いいかえる,動詞-自立,一段,基本形,1
1,と,ト,と,助詞-接続助詞,,,1
2,「,「,「,記号-括弧開,,,1
3,b,b,b,名詞-固有名詞-組織,,,1
4,の,ノ,の,助詞-連体化,,,1


In [6]:
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
s_right_wakati_array = df['strength_right'].progress_apply(get_wakati_text_array)

s_right_wakati_df = create_wakati_df(s_right_wakati_array, 'right_id')

s_right_wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,id
0,α,アルファ,α,記号-アルファベット,,,4
1,と,ト,と,助詞-格助詞-引用,,,4
2,は,ハ,は,助詞-係助詞,,,4
3,「,「,「,記号-括弧開,,,4
4,真実,シンジツ,真実,名詞-一般,,,4


In [7]:
# 今回apply関数をSeries型（text_df['text']）に対して使用するためaxis=1を引数に与えない
w_right_wakati_array = df['weakness_right'].progress_apply(get_wakati_text_array)

w_right_wakati_df = create_wakati_df(w_right_wakati_array, 'right_id')

w_right_wakati_df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,id
0,β,ベータ,β,記号-アルファベット,,,4
1,と,ト,と,助詞-格助詞-引用,,,4
2,は,ハ,は,助詞-係助詞,,,4
3,「,「,「,記号-括弧開,,,4
4,真実,シンジツ,真実,名詞-一般,,,4


In [8]:
s_left_wakati_df = s_left_wakati_df[['original', 'type', 'id']]
w_left_wakati_df = w_left_wakati_df[['original', 'type', 'id']]
s_right_wakati_df = s_right_wakati_df[['original', 'type', 'id']]
w_right_wakati_df = w_right_wakati_df[['original', 'type', 'id']]

s_left_wakati_df['original_type'] = s_left_wakati_df['original'] + '_' + s_left_wakati_df['type']
w_left_wakati_df['original_type'] = w_left_wakati_df['original'] + '_' + w_left_wakati_df['type']
s_right_wakati_df['original_type'] = s_right_wakati_df['original'] + '_' + s_right_wakati_df['type']
w_right_wakati_df['original_type'] = w_right_wakati_df['original'] + '_' + w_right_wakati_df['type']

display(s_left_wakati_df.head(1), w_left_wakati_df.head(1), s_right_wakati_df.head(1), w_right_wakati_df.head(1))

,original,type,id,original_type
0,実,接頭詞-名詞接続,1,実_接頭詞-名詞接続


,original,type,id,original_type
0,いいかえる,動詞-自立,1,いいかえる_動詞-自立


,original,type,id,original_type
0,α,記号-アルファベット,4,α_記号-アルファベット


,original,type,id,original_type
0,β,記号-アルファベット,4,β_記号-アルファベット


In [9]:
s_left_wakati_df = s_left_wakati_df.groupby(['id', 'original_type'])[['type']].count()\
.reset_index().rename(columns={'type':'count'})

w_left_wakati_df = w_left_wakati_df.groupby(['id', 'original_type'])[['type']].count()\
.reset_index().rename(columns={'type':'count'})

s_right_wakati_df = s_right_wakati_df.groupby(['id', 'original_type'])[['type']].count()\
.reset_index().rename(columns={'type':'count'})

w_right_wakati_df = w_right_wakati_df.groupby(['id', 'original_type'])[['type']].count()\
.reset_index().rename(columns={'type':'count'})

display(s_left_wakati_df.head(3), w_left_wakati_df.head(3), s_right_wakati_df.head(3), w_right_wakati_df.head(3))

,id,original_type,count
0,1,a_名詞-一般,1
1,1,a_名詞-固有名詞-組織,3
2,1,b_名詞-一般,1


,id,original_type,count
0,1,a_名詞-一般,1
1,1,a_名詞-固有名詞-組織,1
2,1,b_名詞-固有名詞-組織,2


,id,original_type,count
0,4,α_記号-アルファベット,1
1,4,、_記号-読点,1
2,4,「_記号-括弧開,1


,id,original_type,count
0,4,β_記号-アルファベット,1
1,4,、_記号-読点,1
2,4,「_記号-括弧開,1


In [10]:
s_left_pivot = pd.pivot_table(s_left_wakati_df, index='id', columns='original_type', values='count', fill_value=0)
w_left_pivot = pd.pivot_table(w_left_wakati_df, index='id', columns='original_type', values='count', fill_value=0)
s_right_pivot = pd.pivot_table(s_right_wakati_df, index='id', columns='original_type', values='count', fill_value=0)
w_right_pivot = pd.pivot_table(w_right_wakati_df, index='id', columns='original_type', values='count', fill_value=0)

display(s_left_pivot, w_left_pivot, s_right_pivot, w_right_pivot)

original_type,(_記号-一般,)_記号-一般,00_名詞-数,10%_名詞-固有名詞-一般,12_名詞-数,12人_名詞-固有名詞-一般,20人_名詞-固有名詞-一般,28人_名詞-固有名詞-一般,5%_名詞-固有名詞-一般,8_名詞-数,...,言える_動詞-自立,赤_名詞-一般,赤い_形容詞-自立,部分_名詞-一般,間_名詞-一般,限り_名詞-非自立-副詞可能,際_名詞-非自立-副詞可能,集める_動詞-自立,面積_名詞-一般,領域_名詞-一般
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,2,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,1,1,2,1,0,0,0,2,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


original_type,5%_名詞-固有名詞-一般,a_名詞-一般,a_名詞-固有名詞-組織,b_名詞-固有名詞-組織,α_記号-アルファベット,、_記号-読点,。_記号-句点,「_記号-括弧開,」_記号-括弧閉,あと_名詞-一般,...,計測_名詞-サ変接続,設定_名詞-サ変接続,該当_名詞-サ変接続,説明_名詞-サ変接続,赤い_形容詞-自立,逸脱_名詞-サ変接続,部分_名詞-一般,部隊_名詞-一般,鈍化_名詞-サ変接続,限り_名詞-非自立-副詞可能
id,,,,,,,,,,,,,,,,,,,,,
1,0,1,1,2,1,1,3,5,5,1,...,0,0,0,1,0,2,0,0,0,1
2,1,0,0,1,1,2,1,2,2,0,...,1,1,1,0,1,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


original_type,10%_名詞-固有名詞-一般,15%_名詞-固有名詞-一般,5%_名詞-固有名詞-一般,6%_名詞-固有名詞-一般,=_記号-一般,a_名詞-一般,b_名詞-一般,b_名詞-固有名詞-組織,cvr_名詞-一般,α_記号-アルファベット,...,時_名詞-非自立-副詞可能,期待_名詞-サ変接続,真実_名詞-一般,確率_名詞-一般,組織_名詞-サ変接続,結論_名詞-サ変接続,計測_名詞-サ変接続,赤い_形容詞-自立,部分_名詞-一般,関わる_動詞-自立
id,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,1,0,0,0,1
5,1,1,1,1,2,1,1,1,1,1,...,1,1,2,2,0,1,1,1,1,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


original_type,%_名詞-接尾-助数詞,1%_名詞-固有名詞-一般,10%_名詞-固有名詞-一般,5%_名詞-固有名詞-一般,α_記号-アルファベット,β_記号-アルファベット,、_記号-読点,。_記号-句点,「_記号-括弧開,」_記号-括弧閉,...,真実_名詞-一般,確率_名詞-一般,結論_名詞-サ変接続,表現_名詞-サ変接続,設定_名詞-サ変接続,赤い_形容詞-自立,部隊_名詞-一般,開発_名詞-サ変接続,関わる_動詞-自立,面積_名詞-一般
id,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,1,1,0,1,1,...,1,1,1,0,0,0,0,0,1,0
5,1,1,1,1,3,0,6,3,1,1,...,0,1,1,1,1,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [11]:
'''w_left_pivot, s_right_pivotそれぞれに含まれていない単語を補完'''
w_left_list = w_left_pivot.columns.tolist()
s_right_list = s_right_pivot.columns.tolist()

'''どちらか一方にしかない文字列を抽出'''
diff_set = set(w_left_list)^set(s_right_list)#2つのリストを比較し、重複していない要素のみ抽出
diff_list = list(diff_set)#  set型をlist型に変換
diff_list[0:3]

['全国_名詞-一般', '結果_名詞-副詞可能', '多様_名詞-形容動詞語幹']

In [12]:
'''抽出した重複していない要素のうち、w_left_listに含まれていない要素のみ抽出'''
subtraction_diff_w_left_list= list(set(diff_list) - set(w_left_list))
'''抽出した重複していない要素のうち、s_right_listに含まれていない要素のみ抽出'''
subtraction_diff_s_right_list= list(set(diff_list) - set(s_right_list))

display(len(subtraction_diff_w_left_list), len(subtraction_diff_s_right_list))

28

37

In [13]:
add_w_left_df = pd.DataFrame(
    index = subtraction_diff_w_left_list
)

add_s_right_df = pd.DataFrame(
    index = subtraction_diff_s_right_list
)

add_w_left_df = add_w_left_df.T
add_s_right_df = add_s_right_df.T

display(add_w_left_df.head(), add_s_right_df.head())

,全国_名詞-一般,10%_名詞-固有名詞-一般,として_助詞-格助詞-連語,多様_名詞-形容動詞語幹,同様_名詞-形容動詞語幹,期待_名詞-サ変接続,=_記号-一般,組織_名詞-サ変接続,15%_名詞-固有名詞-一般,強固_名詞-形容動詞語幹,...,b_名詞-一般,なる_動詞-自立,もしも_副詞-助詞類接続,こと_名詞-非自立-一般,cvr_名詞-一般,時_名詞-非自立-副詞可能,6%_名詞-固有名詞-一般,ある_助動詞,関わる_動詞-自立,真実_名詞-一般


,a_名詞-固有名詞-組織,それ_名詞-代名詞-一般,該当_名詞-サ変接続,結果_名詞-副詞可能,そして_接続詞,と_助詞-接続助詞,見る_動詞-自立,的_名詞-接尾-形容動詞語幹,部隊_名詞-一般,いいかえる_動詞-自立,...,ば_助詞-接続助詞,売上_名詞-一般,鈍化_名詞-サ変接続,わけ_名詞-非自立-一般,この_連体詞,限り_名詞-非自立-副詞可能,説明_名詞-サ変接続,た_助動詞,等しい_形容詞-自立,少ない_形容詞-自立


In [14]:
w_left_pivot2 = pd.concat([w_left_pivot, add_w_left_df],axis=1).fillna(0)
s_right_pivot2 = pd.concat([s_right_pivot, add_s_right_df],axis=1).fillna(0)

display(w_left_pivot2, s_right_pivot2)

,5%_名詞-固有名詞-一般,a_名詞-一般,a_名詞-固有名詞-組織,b_名詞-固有名詞-組織,α_記号-アルファベット,、_記号-読点,。_記号-句点,「_記号-括弧開,」_記号-括弧閉,あと_名詞-一般,...,b_名詞-一般,なる_動詞-自立,もしも_副詞-助詞類接続,こと_名詞-非自立-一般,cvr_名詞-一般,時_名詞-非自立-副詞可能,6%_名詞-固有名詞-一般,ある_助動詞,関わる_動詞-自立,真実_名詞-一般
id,,,,,,,,,,,,,,,,,,,,,
1,0,1,1,2,1,1,3,5,5,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,1,1,2,1,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,10%_名詞-固有名詞-一般,15%_名詞-固有名詞-一般,5%_名詞-固有名詞-一般,6%_名詞-固有名詞-一般,=_記号-一般,a_名詞-一般,b_名詞-一般,b_名詞-固有名詞-組織,cvr_名詞-一般,α_記号-アルファベット,...,ば_助詞-接続助詞,売上_名詞-一般,鈍化_名詞-サ変接続,わけ_名詞-非自立-一般,この_連体詞,限り_名詞-非自立-副詞可能,説明_名詞-サ変接続,た_助動詞,等しい_形容詞-自立,少ない_形容詞-自立
id,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,1,1,1,2,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
def cos_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [16]:
w_left_vec = w_left_pivot2.values
s_right_vec = s_right_pivot2.values

w_s_cos_list = [cos_similarity(w_left_vec[i], s_right_vec[i]) for i in range(len(df))]
w_s_cos_list

[0.34544216475888245, 0.4423989983103535, 0.0778498944161523]

In [17]:
df['w_s_cos_sim'] = w_s_cos_list
df

,left_id,strength_left,weakness_left,right_id,strength_right,weakness_right,w_s_cos_sim
0,1,実のところ「AとBに差がない」とは「BはAで無いとは言えない」を意味します。消極的かつ回りく...,いいかえると「Bの結果がAであるにしては逸脱している」かどうかで差の有無を結論するわけです。...,4,αとは「真実として差がないにも関わらず、差があると結論する」偽陽性の確率,βとは「真実として差があるにも関わらず、差がないと結論する」偽陰性の確率,0.345442
1,2,00人がCVR10%の現LPに接触した場合の入会数は20人を中心にばらつきます。そして面積の...,Bの入会数を計測し、その結果がこの赤い部分に該当する場合は「差がある」と結論すると決めたなら...,5,もしもBの真実のCVRが期待してたような15%でなくAと同様の10%である場合、Bの計測後の...,なお、決めたと表現したように、αを何%するかは恣意的です。赤い面積が小さくなるようにαを設定...,0.442399
2,3,在籍メンバーの8割がエンジニア,営業部隊を持っておらず、売上が鈍化,6,強固な営業組織を持ち全国に支社を持つ,開発部隊を持っておらず拡大が急務,0.077850
